In [37]:
import pandas as pd
import foursquare as fs
import requests as re
import sqlite3 as sq
import json

In [22]:
from IPython.display import JSON

#### API keys

In [47]:
import foursquare_env
foursquare_id = foursquare_env.CLIENT_ID
foursquare_secret = foursquare_env.CLIENT_SECRET

In [49]:
import yelp_env
yelp_key = yelp_env.API_KEY

#### Powell's Bookstore in Portland

In [3]:
powells = "45.52354755295152,-122.68132221086628"

#### Results from Foursquare

In [5]:
client = fs.Foursquare(
    client_id=foursquare_id,
    client_secret=foursquare_secret
)

In [13]:
def get_venues(latlong, radius, category_id):
    return client.venues.search(
        dict(
            ll=latlong,
            radius=radius,
            categoryId=category_id,
            limit=50,
        )
    )

In [17]:
bookstore_category_id = "4bf58dd8d48988d114951735"
radius = 2000

In [43]:
# Get new data
# books_fs = get_venues(powells, radius, bookstore_category_id)
# with open('books_fs.json', 'w') as f:
#     json.dump(books_fs, f)

# Recover saved data
with open('books_fs.json', 'r') as f:
    books_fs = json.load(f)

In [44]:
len(books_fs['venues'])

33

Not too many, not too few. Many of these are just parts of the central Powell's location. Also, a *game cafe* somehow snuck in there. We need to clean this up.

In [121]:
books_fs_pd = pd.json_normalize(books_fs['venues']).set_index('id')[
    ['name', 'categories', 'location.address', 'location.lat', 'location.lng']
].rename({
    'location.address': 'address',
    'location.lat': 'latitude',
    'location.lng': 'longitude',
}, axis=1)
books_fs_pd['category'] = books_fs_pd.categories.apply (lambda ls: ls[0]['name'])
books_fs_pd = books_fs_pd.reindex(
    ['name', 'category', 'address', 'latitude', 'longitude'],
    axis=1
)
print(len(books_fs_pd))
# Get rid of all the parts of Powell's
books_fs_pd = books_fs_pd[
    (books_fs_pd.name == "Powell's City of Books") |
    (~books_fs_pd.address.isin(['1005 W Burnside St', "Powell's Bookstore"]))
]
print(len(books_fs_pd))
# Get rid of the wayward gaming cafe
books_fs_pd = books_fs_pd[books_fs_pd.category == 'Bookstore']
books_fs_pd

33
20


,name,category,address,latitude,longitude
id,,,,,
4a06431ff964a520dc721fe3,Powell's City of Books,Bookstore,1005 W Burnside St,45.523180,-122.681375
5d5e132e0948450007d59218,Kinokuniya Bookstore,Bookstore,829 SW 9th Ave,45.518676,-122.681846
4ad3c500f964a52039e620e3,Title Wave Used Bookstore,Bookstore,216 NE Knott St,45.541813,-122.663129
4f70fab9e4b030300bff3f85,Old Oregon Book Store,Bookstore,1017 SW Morrison St,45.520260,-122.682343
4b368e62f964a520e53725e3,Daedalus Books,Bookstore,2074 NW Flanders St,45.525374,-122.694028
4f10b7f4e4b067f66ed5cb02,Mother Foucault's Books,Bookstore,740 SE Morrison St,45.517469,-122.660052
589f7ac472714f7d35788e02,Passages Bookshop,Bookstore,1223 NE M L King Blvd,45.531864,-122.662219
4ff46993e4b09751a655ef99,harmony Market Place,Bookstore,NaN,45.529834,-122.662497
5c2004f82db4a9002ced26b3,Kinokuniya Popup,Bookstore,NaN,45.521960,-122.681220


In [114]:
# Get new data
# book_details_fs = [client.venues(id) for id in books_fs_pd.index]
# with open('book_details_fs.json', 'w') as f:
#     json.dump(book_details_fs, f)

# Recover saved data
with open('book_details_fs.json', 'r') as f:
    book_details_fs = json.load(f)

In [126]:
def get_rating_row(details):
    rating = pd.json_normalize(details['venue']).set_index('id').reindex(
        ['tips.count', 'rating'], axis=1
    ).rename({
        'tips.count': 'review_count'
    }, axis=1)
#     rating = rating.reindex(['review_count', 'rating'], axis=1)
    return rating

In [128]:
ratings = pd.concat(get_rating_row(details) for details in book_details_fs)
ratings

,review_count,rating
id,,
4a06431ff964a520dc721fe3,469,9.4
5d5e132e0948450007d59218,7,8.2
4ad3c500f964a52039e620e3,5,7.2
4f70fab9e4b030300bff3f85,0,NaN
4b368e62f964a520e53725e3,1,7.9
4f10b7f4e4b067f66ed5cb02,2,8.1
589f7ac472714f7d35788e02,0,NaN
4ff46993e4b09751a655ef99,0,NaN
5c2004f82db4a9002ced26b3,0,NaN


In [133]:
books_ratings_fs = books_fs_pd.join(ratings)
del books_ratings_fs['category']
books_ratings_fs

,name,address,latitude,longitude,review_count,rating
id,,,,,,
4a06431ff964a520dc721fe3,Powell's City of Books,1005 W Burnside St,45.523180,-122.681375,469,9.4
5d5e132e0948450007d59218,Kinokuniya Bookstore,829 SW 9th Ave,45.518676,-122.681846,7,8.2
4ad3c500f964a52039e620e3,Title Wave Used Bookstore,216 NE Knott St,45.541813,-122.663129,5,7.2
4f70fab9e4b030300bff3f85,Old Oregon Book Store,1017 SW Morrison St,45.520260,-122.682343,0,NaN
4b368e62f964a520e53725e3,Daedalus Books,2074 NW Flanders St,45.525374,-122.694028,1,7.9
4f10b7f4e4b067f66ed5cb02,Mother Foucault's Books,740 SE Morrison St,45.517469,-122.660052,2,8.1
589f7ac472714f7d35788e02,Passages Bookshop,1223 NE M L King Blvd,45.531864,-122.662219,0,NaN
4ff46993e4b09751a655ef99,harmony Market Place,NaN,45.529834,-122.662497,0,NaN
5c2004f82db4a9002ced26b3,Kinokuniya Popup,NaN,45.521960,-122.681220,0,NaN


#### Results from Yelp

A telling comment from the API documentation: "Note: at this time, the API does not return businesses without any reviews."

In [69]:
lat, long = powells.split(",")
yelp_url = 'https://api.yelp.com/v3'
auth_header = {
    "Authorization": f"Bearer {yelp_key}"
}

In [61]:
def get_yelp_venues(latlong, radius, category):
    lat, long = latlong.split(',')
    response = re.get(
        yelp_url + '/businesses/search',
        params=dict(
            latitude=lat,
            longitude=long,
            radius=radius,
            categories='bookstores',
            limit=50,
        ),
        headers=auth_header
    )
    return response.json()

In [59]:
bookstore_yelp_category = 'bookstores'

In [63]:
# Get new data
# books_yelp = get_yelp_venues(powells, radius, bookstore_yelp_category)
# with open('books_yelp.json', 'w') as f:
#     json.dump(books_yelp, f)

# Recover saved data
with open('books_yelp.json', 'r') as f:
    books_yelp = json.load(f)

In [64]:
len(books_yelp['businesses'])

12

Not as many as Foursquare, but it also seems not to have returned parts of Powell's

In [102]:
books_yelp_pd = pd.json_normalize(books_yelp['businesses']).set_index('id')[
    [
        'name', 'review_count', 'rating',
        'coordinates.latitude', 'coordinates.longitude',
        'location.address1',
    ]
].rename({
    'coordinates.latitude': 'latitude',
    'coordinates.longitude': 'longitude',
    'location.address1': 'address',
}, axis=1)
books_yelp_pd = books_yelp_pd.reindex(
    ['name', 'address', 'latitude', 'longitude', 'review_count', 'rating'],
    axis=1
)
books_yelp_pd

,name,address,latitude,longitude,review_count,rating
id,,,,,,
OPfgKOm_n-ajUo3qjSEgRg,Powell's City of Books,1005 W Burnside St,45.523190,-122.681419,4301,5.0
kROFm3FvMs3AQgqmIg93kg,Kinokuniya Bookstores - Portland,829 SW 9th Ave,45.518731,-122.681943,24,4.5
6XqJYzwxir6fwi3Dw65QhQ,CULT.,1204 NW Glisan St,45.526365,-122.683391,7,5.0
KgKiSBLtA5WwPdGREaOEiw,Daedalus Books,2074 NW Flanders St,45.525370,-122.694080,15,4.5
-5zKNFxuoPm8L9OQ0LtzvA,MudPuddles Toys & Books,2305 NW Kearney St,45.529270,-122.698950,32,4.5
yLATnsDK4UKLVKyy56ac0g,Mother Foucault's Bookshop,523 SE Morrison St,45.517460,-122.660050,30,4.5
YtsrvleoU8bqyScyP9_Tew,OSU Beaver Store,538 SW 6th Ave,45.519850,-122.677790,11,4.5
LALb1wOrPz2LxZTm2emi7Q,Portland State Bookstore,1715 SW 5th Ave,45.511780,-122.681646,21,3.0
c_taCxJnd0MMLNbGTJ3LoQ,Chaparral Books,1975 SW 1st Ave,45.509520,-122.678710,4,4.0


#### Unique Bookstores

Some (well, one) of the bookstores have different names in the two datasets

In [149]:
shared_address = books_ratings_fs.merge(
    books_yelp_pd, on='address', suffixes=('_fs', '_yelp')
)[['name_fs', 'name_yelp']]
shared_address = shared_address[
    shared_address['name_fs'] != shared_address['name_yelp']
]
shared_address

,name_fs,name_yelp
1,Kinokuniya Bookstore,Kinokuniya Bookstores - Portland


In [157]:
books_yelp_rename = books_yelp_pd.merge(
    shared_address, left_on='name', right_on='name_yelp', how='left'
)
books_yelp_rename['name'] = books_yelp_rename['name_fs'].fillna(books_yelp_rename['name'])
del books_yelp_rename['name_fs']
del books_yelp_rename['name_yelp']
books_yelp_rename

,name,address,latitude,longitude,review_count,rating
0,Powell's City of Books,1005 W Burnside St,45.523190,-122.681419,4301,5.0
1,Kinokuniya Bookstore,829 SW 9th Ave,45.518731,-122.681943,24,4.5
2,CULT.,1204 NW Glisan St,45.526365,-122.683391,7,5.0
3,Daedalus Books,2074 NW Flanders St,45.525370,-122.694080,15,4.5
4,MudPuddles Toys & Books,2305 NW Kearney St,45.529270,-122.698950,32,4.5
5,Mother Foucault's Bookshop,523 SE Morrison St,45.517460,-122.660050,30,4.5
6,OSU Beaver Store,538 SW 6th Ave,45.519850,-122.677790,11,4.5
7,Portland State Bookstore,1715 SW 5th Ave,45.511780,-122.681646,21,3.0
8,Chaparral Books,1975 SW 1st Ave,45.509520,-122.678710,4,4.0
9,Passages Bookshop,1223 NE Martin Luther King Jr Blvd,45.531892,-122.661951,2,4.5


In [169]:
bookstores = books_ratings_fs.merge(
    books_yelp_rename, on='name', suffixes=('_fs', '_yelp'), how='outer'
).reindex(['name', 'address_fs', 'address_yelp'], axis=1)
bookstores['address'] = bookstores['address_fs'].fillna(bookstores['address_yelp'])
del bookstores['address_fs']
del bookstores['address_yelp']
bookstores

,name,address
0,Powell's City of Books,1005 W Burnside St
1,Kinokuniya Bookstore,829 SW 9th Ave
2,Title Wave Used Bookstore,216 NE Knott St
3,Old Oregon Book Store,1017 SW Morrison St
4,Daedalus Books,2074 NW Flanders St
5,Mother Foucault's Books,740 SE Morrison St
6,Passages Bookshop,1223 NE M L King Blvd
7,harmony Market Place,NaN
8,Kinokuniya Popup,NaN
9,New Renaissance Bookstore,1338 NW 23rd Ave


#### Database

In [170]:
connection = sq.connect('places.sqlite')

In [171]:
def execute(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
    except sq.Error:
        connection.rollback()
        raise

In [174]:
execute(connection, """
create table places(
    id integer primary key autoincrement,
    name text not null,
    address text
)
"""
)

In [175]:
execute(connection, """
create table review_sites(
    id integer primary key autoincrement,
    name text not null
)
""")

In [176]:
execute(connection, """
create table place_site_entries(
    id integer primary key autoincrement,
    place_id integer,
    site_id integer,
    latitude real,
    longitude real,
    review_count integer,
    rating real,
    foreign key (place_id) references places (id),
    foreign key (site_id) references review_sites (id)
)
""")

In [178]:
bookstores.to_sql('places', connection, if_exists='append', index_label='id')

In [181]:
execute(connection, """
insert into review_sites (name) values ('foursquare'), ('yelp')
""")

In [246]:
pd.read_sql("select * from review_sites", connection)

,id,name
0,1,foursquare
1,2,yelp


In [208]:
def normalize_places(df, site_id):
    df = bookstores.reset_index().merge(
        df, on='name'
    ).rename({'index': 'place_id'}, axis=1)
    df['site_id'] = site_id
    df = df.reindex(
        ['place_id', 'site_id', 'latitude', 'longitude', 'review_count', 'rating'],
        axis=1
    )
    return df

In [210]:
books_fs_db = normalize_places(books_ratings_fs, 1)

In [195]:
books_fs_db.to_sql('place_site_entries', connection, if_exists='append', index=False)

In [212]:
books_yelp_db = normalize_places(books_yelp_pd, 2)

In [213]:
books_yelp_db.to_sql('place_site_entries', connection, if_exists='append', index=False)

#### Ranking

In [247]:
pd.read_sql("""
select distinct site_id, rating from place_site_entries order by site_id, rating
""", connection)

,site_id,rating
0,1,NaN
1,1,7.2
2,1,7.6
3,1,7.9
4,1,8.1
5,1,8.2
6,1,9.4
7,2,3.0
8,2,4.0
9,2,4.5


In [277]:
pd.read_sql("""
select site_id, max(review_count) as max_reviews, max(rating) as max_rating
from place_site_entries
where place_id != 0
group by site_id
""", connection)

,site_id,max_reviews,max_rating
0,1,9,8.2
1,2,32,5.0


The two sites have different ranges of review counts and ratings. Let's multiply the Yelp ratings by two and divide the review counts by three to make them comparable.

In [278]:
reviews = """
select rev.place_id, sites.name,
rev.review_count / case when sites.name = 'yelp' then 3 else 1 end as review_count,
rev.rating * case when sites.name = 'yelp' then 2 else 1 end as rating
from place_site_entries rev
join review_sites sites on sites.id = rev.site_id
"""
pd.read_sql(reviews, connection)

,place_id,name,review_count,rating
0,0,foursquare,469,9.4
1,1,foursquare,7,8.2
2,2,foursquare,5,7.2
3,3,foursquare,0,NaN
4,4,foursquare,1,7.9
5,5,foursquare,2,8.1
6,6,foursquare,0,NaN
7,7,foursquare,0,NaN
8,8,foursquare,0,NaN
9,9,foursquare,9,7.6


Also, many reviews are missing, and we don't want to unduly weight small numbers of reviews. Add some 6/10 reviews to everything and take a weighted average.

In [280]:
dummy_rating = 6.0
dummy_reviews = 2

In [282]:
pd.read_sql(f"""
with reviews as (
{reviews}
)

select p.name,
fs.review_count as fs_reviews, fs.rating as fs_rating,
yelp.review_count as yelp_reviews, yelp.rating as yelp_rating,
(
    coalesce(fs.review_count * fs.rating, 0) +
    coalesce(yelp.review_count * yelp.rating, 0) + {dummy_rating}
) / (coalesce(fs.review_count, 0) + coalesce(yelp.review_count, 0) + {dummy_reviews}) as combined_rating
from places p
left join reviews fs on fs.place_id = p.id and fs.name = 'foursquare'
left join reviews yelp on yelp.place_id = p.id and yelp.name = 'yelp'
order by combined_rating desc
""", connection)

,name,fs_reviews,fs_rating,yelp_reviews,yelp_rating,combined_rating
0,Powell's City of Books,469.0,9.4,1433.0,10.0,9.844853
1,MudPuddles Toys & Books,NaN,NaN,10.0,9.0,8.000000
2,Mother Foucault's Bookshop,NaN,NaN,10.0,9.0,8.000000
3,Daedalus Books,1.0,7.9,5.0,9.0,7.362500
4,Kinokuniya Bookstore,7.0,8.2,NaN,NaN,7.044444
5,New Renaissance Bookstore,9.0,7.6,NaN,NaN,6.763636
6,OSU Beaver Store,NaN,NaN,3.0,9.0,6.600000
7,CULT.,NaN,NaN,2.0,10.0,6.500000
8,Title Wave Used Bookstore,5.0,7.2,NaN,NaN,6.000000
9,Mother Foucault's Books,2.0,8.1,NaN,NaN,5.550000
